In [8]:
query = input('검색어를 입력해주세요. : ')
start_date = input('검색 시작 날짜를 입력해주세요. (형식 : 2022.01.01) : ')
end_date = input('검색 종료 날짜를 입력해주세요. (형식 : 2022.12.31) : ')
sort_type = input('정렬 타입을 입력해주세요. (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : ')
max_page = input('크롤링을 원하는 페이지 수를 입력해주세요. : ')

if start_date > end_date :
    print('\n시작 날짜는 종료 날짜보다 이후로 지정 불가합니다.')
elif max_page == '':
    max_page = 5
    print('\n원하는 페이지 수가 입력되지 않았습니다. 5페이지 까지 크롤링 진행')
    main_crawling(query, start_date, end_date, sort_type, max_page)
else:
    max_page = int(max_page)
    main_crawling(query, start_date, end_date, sort_type, max_page)


1번째 기사글부터 크롤링을 시작합니다.
Processing article : [속보] 경찰 기동대, 이태원 사고 85분 뒤에서야 첫 현장 도착
Processing article : [속보] '이태원 참사' 행안위에 오세훈·김광호·박희영 증인 출석
Processing article : 尹, 이태원 참사 희생자 추모미사 참석…김건희 여사 동행
Processing article : [이태원 참사] 29분 뒤 형사인력 투입…마약단속은 '0건'
Processing article : [속보]尹대통령·김건희 여사, 이태원 참사 추모미사 참석
Processing article : [속보] 윤 대통령 내외 이태원 추모미사 묵상 기도
Processing article : [단독]이태원 참사 '응급의료 컨트롤타워', 결원도 못 채울 판
Processing article : 이태원 참사 '토끼 머리띠' 남성 "얼굴 공개한 사람들 다 고소"
Processing article : [속보] 행안위 "내일 오세훈·김광호·박희영 '이태원 참사' 증인 출석"
Processing article : 이태원 참사 미사 참석한 윤석열 대통령 내외 [포착]

11번째 기사글부터 크롤링을 시작합니다.
Processing article : "윤석열 퇴진이 이태원 추모" 참사 후 첫 촛불집회
Processing article : [속보]尹대통령·김건희 여사, 이태원 참사 추모미사 참석
Processing article : [이태원 참사] 용산서장, '차량이동' 고집해 현장 50분 늦게 도착
Processing article : 이태원 참사 내국인 사망자 130명 장례…오늘 마무리(종합)
Processing article : 이태원 참사 미사 참석한 윤석열 대통령 내외 [포착]
Processing article : 행안위, 내일 이태원 참사 질의…오세훈·김광호·박희영 참석
Processing article : 이태원 참사 후 尹대통령 지지율 추이 보니
Processing article : [이태원 참사] 

In [4]:
import requests
from urllib.request import urlopen

from bs4 import BeautifulSoup # HTTP Response ->   HTML

import pandas as pd
from datetime import datetime
import time
import re
from urllib.parse import quote

In [5]:
def main_crawling(query, start_date, end_date, sort_type, max_page):
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2022.01.01'
    if len(end_date) != 10:
        end_date = '2022.12.31'
    if sort_type not in ['0','1','2']:
        sort_type = '0'
    
    # 각 기사들을 담을 리스트 생성
    titles = []
    dates = []
    articles = []
    article_urls = []
    press_companies = []
    
    # 주어진 일자를 쿼리에 맞는 형태로 변경
    start_date = start_date.replace(".","")
    end_date = end_date.replace(".","")    
    
    # 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링
    current_call = 1
    last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당
    # 1 11 21 31 41....
    
    while current_call <= last_call:
        print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))
        
        url = "https://search.naver.com/search.naver?where=news&query=" + query \
        + "&sort" + sort_type \
        + "nso=so%3Ar%2Cp%3Afrom" + start_date \
        + "to" + end_date \
        + "%2Ca%3A&start=" + str(current_call)
        
        urls_list = []
        
        try:
            web = requests.get(url).content
            source = BeautifulSoup(web,'html.parser')
            for urls in source.find_all('a',{'class':'info'}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    urls_list.append(urls["href"])
        except:
            print("해당 뉴스 검색 페이지의 뉴스 링크를 모으는 중 에러 발생: ", url)
        
        
        if urls_list != []:
            for url in urls_list:
                try:
                    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36where: newssm: tab_jumquery: ' + quote('데이터분석')}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news,'html.parser')
                    # 2) 기사 제목
                    title = source_news.find('h2', {'class':'media_end_head_headline'}).get_text()
                    print('Processing article : {}'.format(title))

                    # 3) 기사 날짜
                    date = source_news.find('span',{'class':'media_end_head_info_datestamp_time'}).get_text()
                    #print('Date of article : {}'.format(date))

                    # 4) 기사 본문
                    article = source_news.find('div',{'id':'dic_area'}).get_text()
                    article = article.replace("\n","")
                    article = article.replace("\t","")

                    # 5) 기사 발행 언론사
                    press_company = source_news.find('em',{'class':'media_end_linked_more_point'}).get_text()

                    # 위 2~5를 통해 성공적으로 제목/날짜/본문/언론사 정보가 모두 추출되었을 때에만 리스트에 추가해 길이를 동일하게 유지
                    titles.append(title)
                    dates.append(date)
                    articles.append(article)
                    press_companies.append(press_company)
                    article_urls.append(url)
                except:
                    print("다음 뉴스기사의 링크를 스크랩 중 오류가 발생하였음 : {}".format(url))
        else:
            pass
        
        time.sleep(5)
        current_call += 10
        
    article_df = pd.DataFrame({'Title':titles,
                            'Data':dates,
                          'Article':articles,
                          'URL':article_urls,
                          'PressCompany':press_companies})
    from openpyxl import Workbook
    article_df.to_excel('result2_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')),index=False,encoding='utf-8')
    article_df.head()